In [1]:
%matplotlib inline
# # # use seaborn for nice default plot settings
import seaborn; seaborn.set_style('ticks')

In [3]:
from astropy.io import fits
import numpy as np

In [36]:
sdssspec = fits.open('spec-1828-53504-0300.fits')
flux = sdssspec[1].data.field('flux')*1e-17
wave = 10**sdssspec[1].data.field('loglam')

u_band = np.genfromtxt('data/u.dat')
g_band = np.genfromtxt('data/g.dat')
r_band = np.genfromtxt('data/r.dat')
i_band = np.genfromtxt('data/i.dat')
z_band = np.genfromtxt('data/z.dat')



In [37]:

def common_wavelength(wlarr_old, fluxarr_old, wlarr_new, fill_value = 0.):
    """
    Function to interpolate flux arrays onto new grid.


    Args:
        wlarr_old (np.array): Input wavelength array to be interpolated.
        wlarr_new (np.array): Target grid
        fluxarr_old (np.array): Flux to be interpolated into target grid
        fill_value (float) = Fill value for values outside interpolation range

    Returns:
        np.array: Resampled flux array

    """
    from scipy import interpolate
    f = interpolate.interp1d(wlarr_old, fluxarr_old, kind='linear', bounds_error = False, fill_value=fill_value)
    fluxarr_new = f(wlarr_new)
    return fluxarr_new

def synth_mag(band=None, wave=None, flux=None):
    """
    Function to calculate synthetic AB magnitudes. Following Bessell & Murphy (2012) and Casagrande & VandenBerg (2014). 


    Args:
        band (np.array): 2D-array containing transmission curve. zero'th column is wavelength in Angstrom and first column is throughput
        wave (np.array): Input wavelength array
        flux (np.array) = Input flux array

    Returns:
        float: Calculated apparent magnitude

    """

    import glob
    import numpy as np
    from astropy.cosmology import Planck13 as cosmo
    from gen_methods import medfilt


    #Read file
    filter_wave = np.array(band[:,0])
    filter_throughput = np.array(band[:,1])


    #Interpolate filter to same sampling as target spectrum
    filt_new =  common_wavelength(filter_wave, filter_throughput, wave, fill_value=0.0)

    #Smooth flux to reject noisy pixels:
    sflux = medfilt(flux, 25)

    #Calculate AB magnitude
    f_nu = (np.sum(filt_new * sflux * wave)) / (3e18 * np.sum(filt_new/wave))
    band_mag = -2.5 * np.log10((f_nu)) - 48.6
    return band_mag


In [38]:
print(synth_mag(u_band, wave, flux))
print(synth_mag(g_band, wave, flux))
print(synth_mag(r_band, wave, flux))
print(synth_mag(i_band, wave, flux))
print(synth_mag(z_band, wave, flux))

16.4534046051
16.982101569
16.8423883441
16.6298281707
16.3265353499
